# Random Forest

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump, load

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn import tree

In [3]:
from preprocessing import reemplazarNulls,reemplazarCategoricas,reemplazarFechas,targetBooleano

[###] All Done                                              


In [4]:
from utilities import score, df_feat, df_targ, train_val

### Codigo a correr

In [5]:
MAX_JOBS = 5

**Run :** Entrenar Nuevamente o Cargar Entrenado

In [6]:
runSimple = True
runValidated = True

**Save :** Guardar Modelo (pisa anterior)

In [7]:
saveSimple = True
saveValidated = True

### Preprocesamiento

In [8]:
targetBooleano(df_targ, inplace=True)
reemplazarNulls(df_feat , inplace=True)
reemplazarCategoricas(df_feat , inplace=True)
reemplazarFechas(df_feat , inplace=True)

,id,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,llovieron_hamburguesas_hoy,mm_evaporados_agua,mm_lluvia_dia,nubosidad_tarde,...,rafaga_viento_max_velocidad,temp_max,temp_min,temperatura_tarde,temperatura_temprano,velocidad_viendo_tarde,velocidad_viendo_temprano,y,m,d
0,74816,247.5,225.0,0.0,78.0,74.0,True,4.0,68.0,7.0,...,48.0,17.9,11.6,17.2,14.8,26.0,17.0,2014,6,7
1,64318,202.5,0.0,0.0,14.0,56.0,True,4.0,5.8,1.0,...,61.0,41.8,22.0,40.2,28.1,31.0,11.0,2014,12,7
2,54357,90.0,90.0,2.6,79.0,47.0,False,4.0,0.0,8.0,...,57.0,19.0,3.1,15.9,13.7,33.0,19.0,2010,7,8
3,1762,337.5,247.5,0.0,92.0,96.0,True,3.0,5.0,8.0,...,54.0,21.5,19.3,19.2,19.4,31.0,13.0,2015,11,4
4,89422,45.0,22.5,0.0,14.0,37.0,False,4.0,0.0,2.0,...,50.0,35.7,18.4,34.8,26.9,19.0,13.0,2014,12,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102351,49272,157.5,90.0,6.5,63.0,44.0,False,3.8,0.4,6.0,...,72.0,20.5,9.6,13.8,14.6,37.0,20.0,2012,5,23
102352,13885,270.0,315.0,2.1,65.0,80.0,True,1.6,5.2,7.0,...,48.0,15.4,8.9,14.3,11.8,31.0,30.0,2011,10,25
102353,4586,247.5,337.5,0.0,81.0,62.0,False,4.0,0.0,7.0,...,35.0,17.9,9.6,15.3,14.6,2.0,6.0,2015,12,16
102354,68030,45.0,67.5,0.9,79.0,65.0,False,2.2,0.0,8.0,...,56.0,12.8,5.9,9.8,9.6,24.0,17.0,2010,8,18


In [9]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102356 entries, 0 to 102355
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            102356 non-null  int64  
 1   direccion_viento_tarde        102356 non-null  float64
 2   direccion_viento_temprano     102356 non-null  float64
 3   horas_de_sol                  102356 non-null  float64
 4   humedad_tarde                 102356 non-null  float64
 5   humedad_temprano              102356 non-null  float64
 6   llovieron_hamburguesas_hoy    102356 non-null  bool   
 7   mm_evaporados_agua            102356 non-null  float64
 8   mm_lluvia_dia                 102356 non-null  float64
 9   nubosidad_tarde               102356 non-null  float64
 10  nubosidad_temprano            102356 non-null  float64
 11  presion_atmosferica_tarde     102356 non-null  float64
 12  presion_atmosferica_temprano  102356 non-nul

In [10]:
train_feat, val_feat, train_targ, val_targ = train_val( df_feat, df_targ )

In [11]:
train_targ = train_targ.llovieron_hamburguesas_al_dia_siguiente
val_targ = val_targ.llovieron_hamburguesas_al_dia_siguiente

## Entrenamiento

### Simple

In [12]:
if (runSimple):
    forest_simple = RandomForestClassifier(
        random_state=1,
        max_depth=3,
        min_samples_split=200,
    )
    forest_simple.fit( train_feat, train_targ )
    
    # Guardar
    if(saveSimple):
        dump(forest_simple, 'models/Ensambles/forest_simple.sk') 
else:
    # Cargar
    forest_simple = load('models/Ensambles/forest_simple.sk')

In [13]:
%time
pred_s = forest_simple.predict(val_feat)
score(pred_s, val_targ)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
ACCURACY: 0.8159437280187574


### Validado

In [14]:
if( runValidated ):
    forest = RandomForestClassifier(
        random_state=1,
    )
    
    params = {
        'max_depth': np.arange(1, 20, 1),
        'min_samples_leaf': np.arange(50, 1000, 50),
        'criterion': ['gini', 'entropy'],
        'n_estimators': np.arange(50,500,50),
    }
    
    forest_grid = HalvingGridSearchCV(
        forest,
        params,
        scoring='accuracy',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1
    ).fit(train_feat, train_targ)
    
    # Guardar
    if(saveValidated):
        dump(forest_grid, 'models/Ensambles/forest_grid.sk')
else:
    # Cargar
    forest_grid = load('models/Ensambles/forest_grid.sk')

n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 42
max_resources_: 92120
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 6498
n_resources: 42
Fitting 5 folds for each of 6498 candidates, totalling 32490 fits
----------
iter: 1
n_candidates: 2166
n_resources: 126
Fitting 5 folds for each of 2166 candidates, totalling 10830 fits
----------
iter: 2
n_candidates: 722
n_resources: 378
Fitting 5 folds for each of 722 candidates, totalling 3610 fits
----------
iter: 3
n_candidates: 241
n_resources: 1134
Fitting 5 folds for each of 241 candidates, totalling 1205 fits
----------
iter: 4
n_candidates: 81
n_resources: 3402
Fitting 5 folds for each of 81 candidates, totalling 405 fits
----------
iter: 5
n_candidates: 27
n_resources: 10206
Fitting 5 folds for each of 27 candidates, totalling 135 fits
----------
iter: 6
n_candidates: 9
n_resources: 30618
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 7
n_candida

In [18]:
print(f"Best score: {forest_grid.best_score_}")
print(f"Best params {forest_grid.best_params_}")

Best score: 0.8472291780076212
Best params {'criterion': 'gini', 'max_depth': 19, 'min_samples_leaf': 50, 'n_estimators': 250}


In [15]:
forest = forest_grid.best_estimator_

In [16]:
%time
pred = forest.predict(val_feat)
score(pred, val_targ)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs
ACCURACY: 0.8403673309886674


In [17]:
dump(forest, 'models/Ensambles/forest.sk')

['models/Ensambles/forest.sk']